In [1]:
import pandas , numpy
import psycopg2
from dotenv import load_dotenv
import os
import time

In [2]:
load_dotenv()

True

In [3]:
#CONNECTING TO COMPANY DB
db_name = "OpenAlex"
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")

In [4]:
conn = psycopg2.connect(f"dbname ={db_name} user={user} password={password} host={host} port={port}")

In [5]:
def Execute_query(query , keyword = None , lst_array_keywords = None):
    """ 
    desired_data:
    author name 
    link to authors profile
    """

    cursor = conn.cursor()
    if lst_array_keywords:
        cursor.execute(query, (lst_array_keywords,))
    elif keyword:
        cursor.execute(query, keyword)
    else:
        cursor.execute(query)

    rows = cursor.fetchall()
    return rows

In [6]:
dict_disease_names = {
    "Cancer": [
        '%cancer%', '%carcinoma%', '%malignancy%', '%neoplasm%',
        '%tumor%', '%tumour%', '%malignant%', '%cancerous%'
    ],
    "Heart disease": [
        '%heart disease%', '%cardiovascular%', '%cardiac%', '%coronary%',
        '%myocardial%', '%angina%', '%congestive heart failure%',
        '%ischemic heart disease%', '%cardiomyopathy%', '%arteriosclerosis%'
    ],
    "Covid": [
        '%covid%', '%covid-19%', '%coronavirus%', '%sars-cov-2%',
        '%2019-ncov%', '%novel coronavirus%', '%coronavirus disease 2019%',
        '%hcov-19%', '%wuhan flu%'
    ],
    "Alzheimer's": [
        '%alzheimer%', '%alzheimers%', '%alzheimer\'s%', '%dementia%',
        '%neurodegenerative%', '%memory loss%', '%cognitive decline%',
        '%senile dementia%', '%presenile dementia%', '%AD%', '%EOAD%', '%YOAD%'
    ],
    "Leukemia": [
        '%leukemia%', '%leukaemia%', '%blood cancer%', '%cancer of the blood%',
        '%hematopoietic cancer%', '%acute leukemia%', '%chronic leukemia%',
        '%lymphocytic leukemia%', '%myeloblastic leukemia%', '%granulocytic leukemia%',
        '%monocytic leukemia%', '%histiocytic leukemia%', '%myeloid leukemia%'
    ],
    "Diabetes": [
        '%diabetes%', '%diabetes mellitus%', '%diabetes insipidus%',
        '%type 1 diabetes%', '%type 2 diabetes%', '%juvenile-onset diabetes%',
        '%adult-onset diabetes%', '%insulin-dependent diabetes%', '%non-insulin-dependent diabetes%',
        '%hyperglycemia%', '%high blood sugar%', '%glucose dysregulation%', '%dm%'
    ],
    "Lymphoma": [
        '%lymphoma%', '%lymphomata%', '%Hodgkin%', '%non-Hodgkin%',
        '%Hodgkin\'s disease%', '%lymphatic malignancy%', '%B-cell lymphoma%',
        '%T-cell lymphoma%', '%follicular lymphoma%', '%Burkitt\'s lymphoma%',
        '%lymphosarcoma%', '%lymphatic neoplasm%'
    ],
    "Sarcoma": [
        '%sarcoma%', '%sarcomas%', '%soft tissue sarcoma%', '%bone sarcoma%',
        '%mesenchymal tumor%', '%liposarcoma%', '%leiomyosarcoma%', '%chondrosarcoma%',
        '%Ewing\'s sarcoma%', '%Kaposi\'s sarcoma%', '%osteosarcoma%',
        '%rhabdomyosarcoma%', '%fibrosarcoma%'
    ],
    "Lung Cancer": [
        '%lung cancer%', '%lung carcinoma%', '%bronchogenic carcinoma%',
        '%pulmonary carcinoma%', '%non-small cell lung cancer%', '%NSCLC%',
        '%small cell lung cancer%', '%SCLC%', '%pulmonary neoplasm%',
        '%bronchial carcinoma%', '%adenocarcinoma of lung%',
        '%squamous cell carcinoma of lung%', '%large cell lung carcinoma%'
    ],
    "Bronchitis": [
        '%bronchitis%', '%acute bronchitis%', '%chronic bronchitis%',
        '%bronchiolitis%', '%bronchial inflammation%', '%inflammation of bronchi%',
        '%respiratory infection%', '%chest cold%', '%bronchial infection%',
        '%cough%', '%bronchospasm%', '%bronchial illness%', '%chest infection%'
    ],
    "Liver Disease": [
        '%liver disease%', '%hepatic disease%', '%liver disorder%', '%hepatic disorder%',
        '%liver dysfunction%', '%hepatic dysfunction%', '%liver failure%',
        '%hepatic insufficiency%', '%cirrhosis%', '%hepatitis%', '%fatty liver%',
        '%hepatic steatosis%', '%non-alcoholic fatty liver disease%', '%NAFLD%',
        '%alcoholic liver disease%', '%MASLD%', '%cholestasis%'
    ]
}

In [ ]:
#getting the top 5 authors
query = """
WITH cancer_related_mesh AS (
    SELECT DISTINCT work_id
    FROM works_mesh
    WHERE descriptor_name ILIKE ANY (ARRAY['%cancer%', '%carcinoma%', '%malignancy%', '%neoplasm%', '%tumor%', '%tumour%', '%malignant%', '%cancerous%'])
)
SELECT
    wa.author_id,
    wa.author_display_name,
    COUNT(DISTINCT w.id) AS works_count,
    COALESCE(SUM(w.cited_by_count), 0) AS total_citations
FROM works w
JOIN cancer_related_mesh crm ON w.id = crm.work_id
JOIN works_authorships wa ON w.id = wa.work_id
GROUP BY
    wa.author_id,
    wa.author_display_name
ORDER BY
    total_citations DESC
LIMIT 5;
"""


In [8]:
cursor = conn.cursor()
cursor.execute(query)
#cursor.execute(query , (["%cancer%"],))

rows = cursor.fetchall()
print(rows)

[('https://openalex.org/A5088652438', 'Saman Warnakulasuriya', 80, Decimal('8816')), ('https://openalex.org/A5112869712', 'Pelayo Correa', 45, Decimal('7000')), ('https://openalex.org/A5017681840', 'B C Morson', 21, Decimal('6439')), ('https://openalex.org/A5016235730', 'Vassilis G. Gorgoulis', 10, Decimal('6393')), ('https://openalex.org/A5055130061', 'Christopher P. Crum', 46, Decimal('6211'))]


In [ ]:
[('https://openalex.org/A5000036388', 'Raymond K. H. Chan', 1, None), ('https://openalex.org/A5000014365', 'Ha Huu Phuoc Dang', 1, None), ('https://openalex.org/A5000022739', 'Marta Gazzaneo', 1, None), ('https://openalex.org/A5000001644', 'Z Pilawski', 1, None), ('https://openalex.org/A5000036901', 'Nicola Creighton', 1, None)]
[('https://openalex.org/A5005301563', 'Alfonso Martin Bernabé', 1, None), ('https://openalex.org/A5007848067', 'Anne-Marie Waser', 1, None), ('https://openalex.org/A5011961771', 'Mathieu Balaguer', 1, None), ('https://openalex.org/A5013101884', 'Jérôme Farinas', 1, None), ('https://openalex.org/A5004327919', 'Christine Le Clainche', 1, None)]



[('https://openalex.org/A5088652438', 'Saman Warnakulasuriya', 80, Decimal('8816')), ('https://openalex.org/A5112869712', 'Pelayo Correa', 45, Decimal('7000')), ('https://openalex.org/A5017681840', 'B C Morson', 21, Decimal('6439')), ('https://openalex.org/A5016235730', 'Vassilis G. Gorgoulis', 10, Decimal('6393')), ('https://openalex.org/A5055130061', 'Christopher P. Crum', 46, Decimal('6211'))]

[('https://openalex.org/A5005301563', 'Alfonso Martin Bernabé', 1, None),
 ('https://openalex.org/A5007848067', 'Anne-Marie Waser', 1, None),
 ('https://openalex.org/A5011961771', 'Mathieu Balaguer', 1, None),
 ('https://openalex.org/A5013101884', 'Jérôme Farinas', 1, None),
 ('https://openalex.org/A5004327919', 'Christine Le Clainche', 1, None)]